In [1]:
import pandas as pd

import utils

bios = pd.read_json("../data/congress-bios.json")
currentBios = bios[bios.congresses.apply(lambda x: 118 in x)]

...what is that at 12395?

In [2]:
display(currentBios["profileText"][12395])

# darn it, Ben Sasse!

str12395 = (
    "a Senator from Nebraska; born in Nebraska City, Nebr., August 19, 1964; graduated"
    " Westside High School, Omaha, Nebr., 1982; B.A., biology, University of Chicago,"
    " 1986; M.B.A., marketing and finance, University of Chicago, 1991; online"
    " brokerage firm president and chief operating officer; co-owner of Chicago Cubs"
    " baseball team; unsuccessful candidate for the United States Senate in 2006;"
    " governor of Nebraska 2015-2023; appointed as a Republican to the United States"
    " Senate on January 12, 2023, to fill the vacancy caused by the resignation of"
    " Benjamin Sasse; took the oath of office on January 23, 2023."
)
currentBios.loc[12395, "profileText"] = str12395

'<!-- [if gte mso 9]><xml>\n <o:OfficeDocumentSettings>\n  <o:AllowPNG/>\n </o:OfficeDocumentSettings>\n</xml><![endif]--><!-- [if gte mso 9]><xml>\n <w:WordDocument>\n  <w:View>Normal</w:View>\n  <w:Zoom>0</w:Zoom>\n  <w:TrackMoves/>\n  <w:TrackFormatting/>\n  <w:PunctuationKerning/>\n  <w:ValidateAgainstSchemas/>\n  <w:SaveIfXMLInvalid>false</w:SaveIfXMLInvalid>\n  <w:IgnoreMixedContent>false</w:IgnoreMixedContent>\n  <w:AlwaysShowPlaceholderText>false</w:AlwaysShowPlaceholderText>\n  <w:DoNotPromoteQF/>\n  <w:LidThemeOther>EN-US</w:LidThemeOther>\n  <w:LidThemeAsian>X-NONE</w:LidThemeAsian>\n  <w:LidThemeComplexScript>X-NONE</w:LidThemeComplexScript>\n  <w:Compatibility>\n   <w:BreakWrappedTables/>\n   <w:SnapToGridInCell/>\n   <w:WrapTextWithPunct/>\n   <w:UseAsianBreakRules/>\n   <w:DontGrowAutofit/>\n   <w:SplitPgBreakAndParaMark/>\n   <w:EnableOpenTypeKerning/>\n   <w:DontFlipMirrorIndents/>\n   <w:OverrideTableStyleHps/>\n  </w:Compatibility>\n  <m:mathPr>\n   <m:mathFont m:val

In [3]:
currentBios = currentBios[
    ["givenName", "familyName", "congresses", "profileText", "jobPositions"]
]

In [ ]:
currentBios.loc[:, "party"] = currentBios.jobPositions.apply(
    lambda x: x[-1]["congressAffiliation"]["partyAffiliation"][-1]["party"]["name"][0],
    # axis=1
)  # ["congress"]["congressNumber"]
currentBios.sample(5)

,givenName,familyName,congresses,profileText,jobPositions,party
4601,Nancy,Pelosi,"[100, 101, 102, 103, 104, 105, 106, 107, 108, ...",a Representative from California; born Nancy D...,"[{'job': {'name': 'Representative', 'jobType':...",D
6374,Blake,Moore,"[117, 118]","a Representative from Utah; born in Ogden, Web...","[{'job': {'name': 'Representative', 'jobType':...",R
5199,Abigail,Spanberger,"[116, 117, 118]",a Representative from Virginia; born in Red Ba...,"[{'job': {'name': 'Representative', 'jobType':...",D
3414,Shelley,Capito,"[107, 108, 109, 110, 111, 112, 113, 114, 115, ...","(Daughter of Arch Alfred Moore, Jr.), a Senato...","[{'job': {'name': 'Senator', 'jobType': 'Congr...",R
8301,Ann,Kuster,"[113, 114, 115, 116, 117, 118]",A Representative from New Hampshire; born in C...,"[{'job': {'name': 'Representative', 'jobType':...",D


In [5]:
def get_state_abbrev(text):
    for state in utils.state_list:
        if state.lower() in text.lower():
            return utils.state_dict[state]

    return None


currentBios.loc[:, "state"] = currentBios.profileText.apply(
    lambda x: get_state_abbrev(x.split(";")[0])
)

In [6]:
currentBios.sample(5)

,givenName,familyName,congresses,profileText,jobPositions,party,state
3912,Tony,Gonzales,"[117, 118]",a Representative from Texas; born on October 1...,"[{'job': {'name': 'Representative', 'jobType':...",R,TX
3941,Anna,Luna,[118],a Representative from Florida; born in Santa A...,"[{'job': {'name': 'Representative', 'jobType':...",R,FL
12480,Jeff,Duncan,"[112, 113, 114, 115, 116, 117, 118]",A Representative from South Carolina; born in ...,"[{'job': {'name': 'Representative', 'jobType':...",R,SC
62,Todd,Young,"[112, 113, 114, 115, 116, 117, 118]",A Senator and a Representative from Indiana; b...,"[{'job': {'name': 'Representative', 'jobType':...",R,IN
4360,Keith,Self,[118],a Representative from Texas; born in Philadelp...,"[{'job': {'name': 'Representative', 'jobType':...",R,TX


### Let's confirm that there are 2 senators from each State:

In [7]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x != 2]

state
NE    3
CA    3
VT    1
OK    1
NC    1
Name: count, dtype: int64

Okay, I assume that the NE and CA anomalies are Ben Sasse retiring and Diane
Feinstein passing away, since the two of them were both members of the 118th
Congress. Let's check on that real quick:

In [8]:
currentBios.loc[
    (currentBios.familyName == "Sasse") | (currentBios.familyName == "Feinstein")
].index

Index([6720, 12640], dtype='int64')

I'm not going to drop either of them, since they were technically parts of the 118th Congress. But that still leaves the question of what's going on in VT, OK, and NC.

In [9]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].loc[lambda x: x.state == "VT"]

,givenName,familyName,congresses,profileText,jobPositions,party,state
5280,Bernard,Sanders,"[102, 103, 104, 105, 106, 107, 108, 109, 110, ...",A Senator and a Representative from Vermont; b...,"[{'job': {'name': 'Representative', 'jobType':...",I,VT


Okay, looks like [Peter Welch](https://en.wikipedia.org/wiki/Peter_Welch) is missing...

In [10]:
currentBios.loc[lambda x: (x.givenName == "Peter") & (x.familyName == "Welch")]

,givenName,familyName,congresses,profileText,jobPositions,party,state
2107,Peter,Welch,"[110, 111, 112, 113, 114, 115, 116, 117, 118]",A Representative from Vermont; born in Springf...,"[{'job': {'name': 'Representative', 'jobType':...",D,VT


I guess they didn't update him in the Congressional Bioguide when he replaced Patrick Leahy? Luckily, this is an easy fix: I'll just do what I did with that massive misencoded html bio from earlier.

In [11]:
currentBios.loc[2107, "profileText"]

'A Representative from Vermont; born in Springfield, Hampden County, Mass., May 2, 1947; graduated from Cathedral High School, Springfield, Mass.; A.B., College of the Holy Cross, Worcester, Mass.; 1969; J.D., University of California, Berkeley, Calif., 1973; lawyer, private practice; member of the Vermont state senate, 1981-1989, 2002-2007, minority leader, 1983-1985, president pro tempore, 1985-1989, 2003-2007; unsuccessful candidate for nomination to the One Hundred First Congress in 1988; elected as a Democrat to the One Hundred Tenth and to the seven succeeding Congresses (January 3, 2007-January 3, 2023); was not a candidate for reelection to the House of Representatives but was elected as a Democrat to the United States Senate in 2022 for the term ending January 3, 2029.'

They even note that he was elected as a Democrat to the Senate at the end of his bio! If I were less lazy, I'd rewrite some code to use "elected as a x to the United States Senate," but I figure so many people get appointed that the way I'm doing it now is easier.

In [12]:
welchBio = (
    "A Senator and a Representative from Vermont; born in Springfield, Hampden County,"
    " Mass., May 2, 1947; graduated from Cathedral High School, Springfield, Mass.;"
    " A.B., College of the Holy Cross, Worcester, Mass.; 1969; J.D., University of"
    " California, Berkeley, Calif., 1973; lawyer, private practice; member of the"
    " Vermont state senate, 1981-1989, 2002-2007, minority leader, 1983-1985, president"
    " pro tempore, 1985-1989, 2003-2007; unsuccessful candidate for nomination to the"
    " One Hundred First Congress in 1988; elected as a Democrat to the One Hundred"
    " Tenth and to the seven succeeding Congresses (January 3, 2007-January 3, 2023);"
    " was not a candidate for reelection to the House of Representatives but was"
    " elected as a Democrat to the United States Senate in 2022 for the term ending"
    " January 3, 2029."
)
currentBios.loc[2107, "profileText"] = welchBio

In [13]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x == 1]

state
OK    1
NC    1
Name: count, dtype: int64

Alright, that's VT down, now just Oklahoma and NC. My bet is that the same thing happened to Markwayne Mullin and Ted Budd.

In [14]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].loc[lambda x: (x.state == "OK") | (x.state == "NC")]

,givenName,familyName,congresses,profileText,jobPositions,party,state
4740,James,Lankford,"[112, 113, 114, 115, 116, 117, 118]",<p>A Senator and a Representative from Oklahom...,"[{'job': {'name': 'Representative', 'jobType':...",R,OK
11543,Thomas,Tillis,"[114, 115, 116, 117, 118]",A Senator from North Carolina; born in Jackson...,"[{'job': {'name': 'Senator', 'jobType': 'Congr...",R,NC


Yep, my hunch was right.

In [15]:
display(
    currentBios.loc[lambda x: (x.givenName == "Markwayne") & (x.familyName == "Mullin")]
)
currentBios.loc[11757, "profileText"] = currentBios.loc[11757, "profileText"].replace(
    "a Representative from", "a Senator and a Representative from"
)

display(
    currentBios.loc[lambda x: (x.givenName == "Markwayne") & (x.familyName == "Mullin")]
)

,givenName,familyName,congresses,profileText,jobPositions,party,state
11757,Markwayne,Mullin,"[113, 114, 115, 116, 117, 118]","a Representative from Oklahoma; born in Tulsa,...","[{'job': {'name': 'Representative', 'jobType':...",R,OK


,givenName,familyName,congresses,profileText,jobPositions,party,state
11757,Markwayne,Mullin,"[113, 114, 115, 116, 117, 118]",a Senator and a Representative from Oklahoma; ...,"[{'job': {'name': 'Representative', 'jobType':...",R,OK


In [16]:
display(
    currentBios.loc[lambda x: (x.givenName == "Theodore") & (x.familyName == "Budd")]
)
currentBios.loc[9656, "profileText"] = currentBios.loc[9656, "profileText"].replace(
    "a Representative from", "a Senator and a Representative from"
)

display(
    currentBios.loc[lambda x: (x.givenName == "Theodore") & (x.familyName == "Budd")]
)

,givenName,familyName,congresses,profileText,jobPositions,party,state
9656,Theodore,Budd,"[115, 116, 117, 118]",a Representative from North Carolina; born in ...,"[{'job': {'name': 'Representative', 'jobType':...",R,NC


,givenName,familyName,congresses,profileText,jobPositions,party,state
9656,Theodore,Budd,"[115, 116, 117, 118]",a Senator and a Representative from North Caro...,"[{'job': {'name': 'Representative', 'jobType':...",R,NC


In [17]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x == 1]

Series([], Name: count, dtype: int64)

Alright, now we can easily flag everyone who was a senator in the 118th congress! Let's make an indicator real quick:

In [18]:
currentBios["senator"] = (
    currentBios.profileText.str.lower().str.contains("a senator from")
    | currentBios.profileText.str.lower().str.contains("a senator and a represent")
    | currentBios.profileText.str.lower().str.contains("a senator and represent")
).astype(bool)

### Checking representatives

In [19]:
currentBios.senator.apply(lambda x: not x).sum()

np.int64(441)

Okay, this is what I expected! There are, after all, [6 non-voting members of the House of Representatives](https://en.wikipedia.org/wiki/Non-voting_members_of_the_United_States_House_of_Representatives), and 435 = 441 - 6. I don't really care about the non-voting members, though (no offense), so I'll be dropping them (as well as making sure each state has the correct number of reps).

I might want to write a more robust script to handle Congresses before 1960 (i.e. more robust "was this state a state at the time" handling), but that isn't a problem right now.

In [20]:
currentBios.loc[currentBios.senator.apply(lambda x: not x) & currentBios.state.isna()]

,givenName,familyName,congresses,profileText,jobPositions,party,state,senator
2306,Aumua,Radewagen,"[114, 115, 116, 117, 118]",a Delegate from American Samoa; born in Washin...,"[{'job': {'name': 'Delegate', 'jobType': 'Cong...",R,None,False
2586,Stacey,Plaskett,"[114, 115, 116, 117, 118]",A Delegate from the Virgin Islands; born in Ne...,"[{'job': {'name': 'Delegate', 'jobType': 'Cong...",D,None,False
3087,James,Moylan,[118],"a Delegate from Guam; born on July 18, 1962; g...","[{'job': {'name': 'Delegate', 'jobType': 'Cong...",R,None,False
3812,Jenniffer,González-Colón,"[115, 116, 117, 118]",a Resident Commissioner from Puerto Rico; born...,"[{'job': {'name': 'Resident Commissioner', 'jo...",R,None,False
5134,Gregorio,Sablan,"[111, 112, 113, 114, 115, 116, 117, 118]",a Delegate from the Northern Mariana Islands; ...,"[{'job': {'name': 'Delegate', 'jobType': 'Cong...",D,None,False
12811,Eleanor,Norton,"[102, 103, 104, 105, 106, 107, 108, 109, 110, ...",a Delegate from the District of Columbia; born...,"[{'job': {'name': 'Delegate', 'jobType': 'Cong...",D,None,False


In [21]:
currentBios.drop(
    currentBios.loc[
        currentBios.senator.apply(lambda x: not x) & currentBios.state.isna()
    ].index,
    inplace=True,
)

In [22]:
reps = {
    get_state_abbrev(k): v
    for k, v in zip(utils.number_of_reps.keys(), utils.number_of_reps.values())
}

In [ ]:
[
    state
    for state in currentBios.loc[currentBios.senator.apply(lambda x: not x)]
    .state.value_counts()
    .index
    if currentBios.loc[currentBios.senator.apply(lambda x: not x)]
    .state.value_counts()
    .loc[state]
    != reps[state]
]

In [23]:
currentBios.loc[
    currentBios.senator.apply(lambda x: not x)
].state.value_counts()  # .sum()

state
CA    52
TX    38
FL    28
NY    26
PA    17
IL    17
OH    15
GA    14
NC    14
MI    13
NJ    12
VA    11
WA    10
AZ     9
TN     9
MA     9
IN     9
MO     8
WI     8
MN     8
MD     8
CO     8
SC     7
AL     7
OR     6
LA     6
KY     6
OK     5
CT     5
KS     4
UT     4
NV     4
MS     4
AR     4
IA     4
NM     3
NE     3
ID     2
NH     2
HI     2
WV     2
ME     2
RI     2
MT     2
SD     1
WY     1
AK     1
VT     1
ND     1
DE     1
Name: count, dtype: int64